# Answering Business Questions using SQL
In this project, we're going to practice using our SQL skills to answer business questions. Our starting point is a ficticious company called `Chinook` which sells music to customers. As an analyst, we might be given access to a database with tables associated with employees, customers, tracks/albums, transactions, and more. We will simulate this with the local database `chinook.db` which has the following schema diagram:

https://s3.amazonaws.com/dq-content/191/chinook-schema.svg

Let's start by connecting Jupyter to `chinook.db` so we can start exploring:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
# List of tables and views:

In [4]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [10]:
# Simple list of artists

In [9]:
%%sql

SELECT *
  FROM artist
 LIMIT 5;

 * sqlite:///chinook.db
Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


## Selecting albums to purchase for Chinook 
Imagine that the Chinook store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre   |
|------|------|
|   Regal  | Hip-Hop |
|   Red Tone | Punk |
|   Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

Further, suppose that this label represents artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [22]:
%%sql

WITH 
    usa_invoices AS
    (
        SELECT *
          FROM invoice
         INNER JOIN invoice_line AS inv_line ON inv_line.invoice_id = invoice.invoice_id
         WHERE billing_country = 'USA'
    ),
    track_and_genre AS
    (
        SELECT 
            track.track_id,
            genre.name AS genre
          FROM track
         INNER JOIN genre ON genre.genre_id = track.genre_id
    )
    
    
SELECT 
    tng.genre AS genre,
    SUM(us_inv.quantity) AS tracks_sold,
    CAST(SUM(us_inv.quantity) AS FLOAT) / CAST(
        (SELECT SUM(quantity) FROM usa_invoices) AS FLOAT) AS percentage_sold
  FROM usa_invoices AS us_inv
 INNER JOIN track_and_genre AS tng ON tng.track_id = us_inv.track_id
 GROUP BY tng.genre
 ORDER BY tracks_sold DESC
 LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


From the above results we might recommend purchasing the sale rights for the Punk group and advise against purchasing the rights for the Hip-Hop and Pop artists. We might also note that Rock is by far and away our most popular genre.

On the other hand, we might also be suggest exploring the quantity of artists for each genre currently listed in the store in the in
